In [2]:
import numpy as np

In [26]:
Lx, Ly, Lz = 1, 1, 1
nodes = 8
hx = Lx / (nodes - 1)
hy = Ly / (nodes - 1)
hz = Lz / (nodes - 1)


time_ticks = 10
tau = 1e-2

X = np.arange(nodes)[:, np.newaxis, np.newaxis, np.newaxis] / (nodes - 1) * Lx
Y = np.arange(nodes)[np.newaxis, :, np.newaxis, np.newaxis] / (nodes - 1) * Ly
Z = np.arange(nodes)[np.newaxis, np.newaxis, :, np.newaxis] / (nodes - 1) * Lz
T = np.arange(time_ticks)[np.newaxis, np.newaxis, np.newaxis, :] * tau

analytical = np.sin(np.pi * X / Lx) * np.sin(np.pi * Y / Ly) * np.sin(np.pi * Z / Lz) * np.cos(3 * T) 

# analytical = np.sin(3 * np.pi * X / Lx) * np.sin( 3 * np.pi * Y / Ly) * np.sin(np.pi * Z / Lz) * np.cos(4 * (T + np.pi)) 


In [31]:
T

array([[[[0.000000, 0.010000, 0.020000, 0.030000, 0.040000, 0.050000,
          0.060000, 0.070000, 0.080000, 0.090000]]]])

In [30]:
print(analytical[:,:,:,1])

[[[0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
   0.000000]
  [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
   0.000000]
  [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
   0.000000]
  [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
   0.000000]
  [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
   0.000000]
  [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
   0.000000]
  [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
   0.000000]
  [0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
   0.000000]]

 [[0.000000 0.000000 0.000000 0.000000 0.000000 0.000000 0.000000
   0.000000]
  [0.000000 0.081644 0.147118 0.183453 0.183453 0.147118 0.081644
   0.000000]
  [0.000000 0.147118 0.265097 0.330570 0.330570 0.265097 0.147118
   0.000000]
  [0.000000 0.183453 0.330570 0.412214 0.412214 0.330570 0.183453
   0.000000]
  [0.000000 0.183453 0.330570 0.412214 0.412214 0.

In [40]:
def laplasian(a):
    padded = np.pad(a, 1, 'constant')
    out = (padded[:-2,1:-1,1:-1] - 2 * padded[1:-1,1:-1,1:-1] + padded[2:,1:-1,1:-1]) / hx ** 2 + \
          (padded[1:-1,:-2,1:-1] - 2 * padded[1:-1,1:-1,1:-1] + padded[1:-1,2:,1:-1]) / hy ** 2 + \
          (padded[1:-1,1:-1,:-2] - 2 * padded[1:-1,1:-1,1:-1] + padded[1:-1,1:-1,2:]) / hz ** 2
    
    return out

def process_borders(a):
    a[0,:,:] = 0
    a[-1,:,:] = 0
    a[:,0,:] = 0
    a[:,-1,:] = 0
    a[:,:,0] = 0
    a[:,:,-1] = 0

result = np.zeros((nodes, nodes, nodes, time_ticks))
# result[:,:,:,0:1] = np.sin(np.pi * X / Lx) * np.sin(np.pi * Y / Ly) * np.sin(np.pi * Z / Lz)
# result[:,:,:,1] = result[:,:,:,0] + tau ** 2 * laplasian(result[:,:,:,0]) / 2

result[:,:,:,0] = analytical[:,:,:,0]
result[:,:,:,1] = analytical[:,:,:,1]

np.set_printoptions(precision=6)
np.set_printoptions(floatmode='fixed')

# print(analytical[:,:,:,0])

for i in range(2, time_ticks):
    lapl = laplasian(result[:,:,:,i-1])
    result[:,:,:,i] = 2 * result[:,:,:,i-1] - result[:,:,:,i-2] + tau ** 2 * lapl / 2
    process_borders(result[:,:,:,i])
    print(np.max(abs(result[:,:,:,i] - analytical[:,:,:,i])), np.max(abs(analytical[:,:,:,i] - analytical[:,:,:,i-1])))



0.0005148257220984886 0.0012505135770334297
0.0015430326379324288 0.0020829388377434643
0.0030819661776559926 0.0029134895940936545
0.005127766636247588 0.0037414184064636924
0.007675376249647092 0.00456598019480603
0.010718548429111019 0.005386432909167738
0.01424985913865906 0.0062020381974843275
0.018260720396991892 0.007012062070048231
